<a href="https://colab.research.google.com/github/jhmuller/mlp-covid19/blob/main/mlp_covid19_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
from __future__ import unicode_literals, print_function, division
import unicodedata
import string
import sys
import os
import re
import random
from tqdm import tqdm
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
def whoami(): 
    return sys._getframe(1).f_code.co_name
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
# need to intall this every time on colab, not sure why but I do it
import importlib
import warnings
import pdb
import datetime
now = datetime.datetime.now
def warning_on_one_line(message, category, filename, lineno, file=None, line=None):
    return 'line:%s  cat:<%s>  msg:%s\n'% (lineno, category.__name__, message)  

warnings.formatwarning = warning_on_one_line

def global_imports(imports=None, verbosity=0):
  fails = []
  if not imports:
    return
  if not isinstance(imports, list):
    msg = "imports shoule be a list with elements like\m"
    msg += "'foo' or 'foo as bar'"
    warnings.warn(msg)
    return
  for line in imports:
    parts = line.split()
    if len(parts) not in [1,3] or \
          (len(parts) == 3 and parts[1] != 'as'):
      msg = "<{0}> not a valid import line\n"
      msg += "use 'import foo' or 'import foo as bar'"
      warnings.warn(msg)
      continue
    module_name = parts[0]
    asname = parts[0]
    if len(parts) == 3:
      asname = parts[2]
    try:
      module = importlib.import_module(module_name)
      globals()[asname] = module
    except Exception as e:
      print(e)
      print("{0} not installed\Trying to install it".format(module_name))
      try:
        !pip install $module_name
        module = importlib.import_module(module_name)  
        globals()[asname] = module        
      except Exception as e2:
        print(e2)
        fails.append(module_name)
  if fails:
    msg = "Not able to import or install\n{0}".format(fails)
  else:
    msg = "Success. imported {0}".format(imports)
  if verbosity > 0:
    print(msg)
  return fails
    
def is_defined(varnames=None, verbosity=0):
  if isinstance(varnames, str):
    varnames = [varnames]
  for v in varnames:
    try:
      eval(v)
      if verbosity > 0:
        print("'{0}'' is defined")
    except Exception as e:
      warnings.warn("trying to eval '{0}'".format(v))
      print(e)

imports = [ 'fsspec',
           'nltk',
            'pandas as pd',
            'numpy as np']
varnames = ['fsspec', 'pd']
global_imports(imports, verbosity=1)
# check if the imports worked
is_defined([x.split()[-1] for x in imports])

Success. imported ['fsspec', 'nltk', 'pandas as pd', 'numpy as np']


In [33]:
import time
import math


def asMinutes(s):
  m = math.floor(s / 60)
  s -= m * 60
  return '%dm %ds' % (m, s)


def timeSince(since, percent):
  now = time.time()
  s = now - since
  es = s / (percent)
  rs = es - s
  return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [34]:
# I am *guessing* this is max_len_text from part 1
max_length_text = 600


In [35]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  # to mount google drive for the input files
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=False)  
  dpath = "/content/gdrive/My Drive/mlp_covid19/data/part1_output"  
else:
  print('Not running on CoLab')
  dpath = "./mlp_covid19/data/part1_output"
if not os.path.isdir(dpath):
  msg = "{0} is not a path".format(dpath)
  warnings.warn(msg)
else:
  files = os.listdir(dpath)
  msg = "{0} has {1} files".format(dpath, len(files))
  n2print = 4
  suffix = "..." if len(files) > n2print else ""
  fsizes =[os.path.getsize(os.path.join(dpath, fname)) for fname in files ]
  for i, f in enumerate(files):
    msg += "\n\t'{0}' size: {1} MB  {2} GB".format(f, np.round(fsizes[i]/10**6,2), 
                                             np.round(fsizes[i]/10**9, 3))
  print(msg)

Running on CoLab
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/mlp_covid19/data/part1_output has 4 files
	'pairs_df_part1.csv' size: 123.59 MB  0.124 GB
	'input_lang_part1.parquet' size: 141.22 MB  0.141 GB
	'output_lang_part1.parquet' size: 6.41 MB  0.006 GB
	'pairs_part1.parquet' size: 124.92 MB  0.125 GB


In [36]:
SOS_token = 0
EOS_token = 1
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [44]:
def load_path_objects(dpath, 
                      force_reload=True,
                      extensions=("parquet", "csv"),
                      verbosity=0):
  if verbosity > 0:
    print("\n{0} force_reload= {1}, extensions={2}".format(whoami(),
                                                        force_reload,
                                                         extensions))  
  import pickle
  if not extensions:
    warnings.warn("No extensions")
    return None
  res = {}  
  known_exts = {"parquet", "csv"}
  all_files = [fname for fname in os.listdir(dpath)]
  all_exts = set([file.split(".")[-1] for file in all_files])
  unknown_exts = all_exts.difference(known_exts)
  if len(unknown_exts) > 0:
    if verbosity > 0:
      known_str = ",".join(known_extensions)
      msg = "Can only handle these extensions: [{0}]".format(known_str)
      unknown_str = ",".join(unknown_exts)
      msg += "  skipping files with exts in [{0}]".format(unknown_str)
      print(msg)
  files = [f for f in all_files if f.split(".")[-1] in known_exts]
  if verbosity > 0:
    n = 6
    suff = "..." if len(files) > n else ""    
    print("\nloading files: [{0}{1}]".format(",".join(files[:n]), suff))
    print(" from: {0}".format(dpath))

  for i, file in enumerate(tqdm(files, mininterval=0.01, smoothing=0.1)):
    if verbosity > 0:
      print("{0} '{1}'".format(i, file))
    parts = file.split(".")
    if len(parts) != 2:
      print("Cannot do {0}, exptecing format 'name.ext'".format(file))
      continue
    obj_name, ext = parts
    if verbosity > 0:
      print("obj_name: '{0}', ext= '.{1}'".format(obj_name, ext))
    fpath = os.path.join(dpath, file)
    if not os.path.isfile(fpath):
      warnings.warn("{0} is not a filepath".format(fpath))
      continue
    if verbosity > 0:
      print("\t{0} from {1}".format(obj_name,file))    
    if not force_reload:
      if obj_name in globals().keys() :
        print("{0} already in globals, not reloading".format(obj_name))
        continue
      else:
        print("{0} not in globals, must load".format(obj_name))        

    res[obj_name] = None
    try:
      if ext == "parquet":
        with open(fpath, mode='rb') as file:
          obj = pickle.load(file)
          res[obj_name] = obj
      elif ext == 'csv':
        obj = pd.read_csv(fpath) 
    except Exception as e:
      warnings.warn("  *Read Failed* on {0} ".format(file, dpath))
      warnings.warn(str(e))
    else:
      res[obj_name] = obj
      print("  OK {0} is type {1}".format(obj_name, 
                                          type(res[obj_name])))
  return res
res = load_path_objects(dpath=dpath, force_reload=False, verbosity=2)
for k in res.keys():
  obj = res[k]
  print("adding {0} type{1} to global namespace".format(k, type(obj)))
  globals()[k] = res[k]

100%|██████████| 4/4 [00:00<00:00, 691.02it/s]


load_path_objects force_reload= False, extensions=('parquet', 'csv')

loading files: [pairs_df_part1.csv,input_lang_part1.parquet,output_lang_part1.parquet,pairs_part1.parquet]
 from: /content/gdrive/My Drive/mlp_covid19/data/part1_output
0 'pairs_df_part1.csv'
obj_name: 'pairs_df_part1', ext= '.csv'
	pairs_df_part1 from pairs_df_part1.csv
pairs_df_part1 already in globals, not reloading
1 'input_lang_part1.parquet'
obj_name: 'input_lang_part1', ext= '.parquet'
	input_lang_part1 from input_lang_part1.parquet
input_lang_part1 already in globals, not reloading
2 'output_lang_part1.parquet'
obj_name: 'output_lang_part1', ext= '.parquet'
	output_lang_part1 from output_lang_part1.parquet
output_lang_part1 already in globals, not reloading
3 'pairs_part1.parquet'
obj_name: 'pairs_part1', ext= '.parquet'
	pairs_part1 from pairs_part1.parquet
pairs_part1 already in globals, not reloading


In [45]:
len(pairs_part1)

62357

In [46]:
lendf = pd.DataFrame(pairs_df_part1['X'].apply(lambda x: len(x)))
#lendf.plot.hist(bins=1000)
cols = [len(x) < 4 for x in pairs_df_part1['X'].values]
pairs_df_part1.loc[cols]
#lendf.head()
#pairs_df.head()

,X,Y
20508,L>,"Coronavirus, ecco i sintomi: sono gli stessi d..."
28568,E>,"Coronavirus cinese, Oms corregge valutazione: ..."
58388,E>,"Coronavirus, donna di 54 anni sulla nave da cr..."


In [47]:
'''
Reusable set of functions to convert a tuple of strings (pair) to tensors
Reference: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
'''
if True:
  import torch

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


  def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


  def tensorsFromPair(pair, input_lang, output_lang):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [48]:
class Encoder(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(Encoder, self).__init__()
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)

  def initHidden(self):
    self.hidden = torch.zeros(1, 1, self.hidden_size, device=device)

  def forward(self, input, hidden):
    embedded = self.embedding(input).view(1, 1, -1)
    output = embedded
    output, hidden = self.gru(output, hidden)
    return output, hidden

In [49]:
class EncoderRNN(nn.Module):
  def __init__(self, input_size, hidden_size, verbosity=0):
    super(EncoderRNN, self).__init__()
    self.hidden_size = hidden_size
    self.verbosity = verbosity
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)

  def forward(self, input, hidden):
    if self.verbosity > 0:
      print("{0} input.shape {1}  hidden.shape {2}".format(whoami(),
                                                           input.shape, hidden.shape))
    embedded = self.embedding(input).view(1, 1, -1)
    output = embedded
    output, hidden = self.gru(output, hidden)
    return output, hidden

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [50]:
class DecoderRNN(nn.Module):
  def __init__(self, hidden_size, output_size):
    super(DecoderRNN, self).__init__()
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(output_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, input, hidden):
    output = self.embedding(input).view(1, 1, -1)
    output = F.relu(output)
    output, hidden = self.gru(output, hidden)
    output = self.softmax(self.out(output[0]))
    return output, hidden

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [51]:
class AttnDecoderRNN(nn.Module):
  def __init__(self, hidden_size, output_size, max_length, dropout_p=0.1,
               verbosity=0):
    super(AttnDecoderRNN, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.dropout_p = dropout_p
    self.max_length = max_length
    self.verbosity = verbosity
    self.embedding = nn.Embedding(self.output_size, self.hidden_size)
    self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
    self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
    self.dropout = nn.Dropout(self.dropout_p)
    self.gru = nn.GRU(self.hidden_size, self.hidden_size)
    self.out = nn.Linear(self.hidden_size, self.output_size)

  def forward(self, input, hidden, encoder_outputs):
    if self.verbosity > 0:
      print("{0} ".format(whoami()))  
      print("input type {0}".format(type(input)))  
      print("hidden type {0}".format(type(hidden)))        
    embedded = self.embedding(input).view(1, 1, -1)
    embedded = self.dropout(embedded)

    attn_weights = F.softmax(
        self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
    if self.verbosity > 0:
      print("  attn_weights.shape: {0}".format(attn_weights.shape))
      print("  encoder_outputs.shape: {0}".format(encoder_outputs.shape))      
    attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                              encoder_outputs.unsqueeze(0))

    output = torch.cat((embedded[0], attn_applied[0]), 1)
    output = self.attn_combine(output).unsqueeze(0)

    output = F.relu(output)
    output, hidden = self.gru(output, hidden)

    output = F.log_softmax(self.out(output[0]), dim=1)
    return output, hidden, attn_weights

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [52]:
def trainIters(encoder, decoder, n_iters, 
               max_length, 
               input_lang, output_lang, pairs,
               print_every=1000, plot_every=100, 
               learning_rate=0.01, verbosity = 0):
  if verbosity > 0:
    print("{0} n_iters: {1}".format(whoami(), n_iters))
  start_dt = datetime.datetime.now()
  plot_losses = []
  print_loss_total = 0  # Reset every print_every
  plot_loss_total = 0  # Reset every plot_every  

  encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
  decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
  training_pairs = [tensorsFromPair(pair=random.choice(pairs), 
                                    input_lang=input_lang,
                                    output_lang=output_lang) for i in range(n_iters)]
  criterion = nn.NLLLoss()

  if verbosity > 0:
    print("Start iter dt:{0}".format(start_dt))
  losses = []
  for iter in tqdm(range(0, n_iters), mininterval=0.1, smoothing=0.25):
    if verbosity > 0:
      if iter % print_every == 0:
        print(iter+1,"/",n_iters)
    training_pair = training_pairs[iter]
    input_tensor = training_pair[0]
    target_tensor = training_pair[1]

    loss = train(input_tensor=input_tensor, target_tensor=target_tensor, 
                 max_length=max_length,
                 encoder=encoder, decoder=decoder, 
                 encoder_optimizer=encoder_optimizer, 
                 decoder_optimizer=decoder_optimizer, 
                 criterion=criterion,
                 verbosity=verbosity) 
    if verbosity > 0:  
      if iter % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                      iter, iter / n_iters * 100, 
                                     print_loss_avg))

    if iter % plot_every == 0:
      plot_loss_avg = plot_loss_total / plot_every
      plot_losses.append(plot_loss_avg)
      plot_loss_total = 0
  
  showPlot(plot_losses)      

def  train(input_tensor, target_tensor, encoder, decoder, 
           encoder_optimizer, decoder_optimizer, 
           criterion, max_length, 
           verbosity=0):
  if verbosity > 0:
    print("{0}".format(whoami() ))
    print("  input_tensor.shape {0}".format(input_tensor.shape))
    print("  target_tensor.shape: {0}".format(target_tensor.shape))
  encoder_hidden = encoder.initHidden()

  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()

  input_length = input_tensor.size(0)
  target_length = target_tensor.size(0)

  encoder_outputs = torch.zeros(input_length, encoder.hidden_size, device=device)
  loss = 0
  for i in tqdm(range(input_length)):
    encoder_output, encoder_hidden = encoder(input_tensor[i], 
                                             encoder_hidden)
    encoder_outputs[i] = encoder_output[0, 0]
  
  decoder_input = torch.tensor([[SOS_token]], device=device)
  
  decoder_hidden = encoder_hidden

  teacher_forcing_ratio = 0.5
  coin = np.round(random.random(), 3)
  use_teacher_forcing = True if coin < teacher_forcing_ratio else False
  if verbosity > 0:
    msg = "use_teacher_forcing: {0}".format(use_teacher_forcing)
    msg += "\n\tcoin: {0} teacher_forcing_ratio: {1}".format(coin, 
                                                         teacher_forcing_ratio)
    print(msg)

  if use_teacher_forcing:
    # Teacher forcing: Feed the target as the next input
    for di in range(target_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(
           decoder_input, decoder_hidden, encoder_outputs)
      loss += criterion(decoder_output, target_tensor[di])
      decoder_input = target_tensor[di]  # Teacher forcing

  else:
    # Without teacher forcing: use its own predictions as the next input
    for di in range(target_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(
          decoder_input, decoder_hidden, encoder_outputs)
      topv, topi = decoder_output.topk(1)
      decoder_input = topi.squeeze().detach()  # detach from history as input

      loss += criterion(decoder_output, target_tensor[di])
      if decoder_input.item() == EOS_token:
        break

  loss.backward()

  encoder_optimizer.step()
  decoder_optimizer.step()  

  return loss.item / target_length



In [53]:
def evaluate(encoder, decoder, sentence, max_length,
             verbosity=0):
  with torch.no_grad():
    input_tensor = tensorFromSentence(input_lang, sentence)
    input_length = input_tensor.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    if verbosity > 0:
      print("encoder outputs.shape {0}".format(encoder_outputs.shape))
      print("input_length= {0} initialing encoder input".format(input_length))
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                  encoder_hidden)
        if verbosity > 0:
          print("{0} type ".format(type(encoder_output)))
        encoder_outputs[ei] += encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

    decoder_hidden = encoder_hidden

    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)

    for di in range(max_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(
          decoder_input, decoder_hidden, encoder_outputs)
      decoder_attentions[di] = decoder_attention.data
      topv, topi = decoder_output.data.topk(1)
      if topi.item() == EOS_token:
        decoded_words.append('<EOS>')
        break
      else:
        decoded_words.append(output_lang.index2word[topi.item()])

        decoder_input = topi.squeeze().detach()

    return decoded_words, decoder_attentions[:di + 1]

In [54]:
def evaluateRandomly(encoder, decoder, n=10):
  for i in range(n):
    pair = random.choice(pairs)
    print('>', pair[0])
    print('=', pair[1])
    output_words, attentions = evaluate(encoder, decoder, pair[0])
    output_sentence = ' '.join(output_words)
    print('<', output_sentence)
    print('')

In [55]:
######################################################################
# We'll need a unique index per word to use as the inputs and targets of
# the networks later. To keep track of all this we will use a helper class
# called ``Lang`` which has word → index (``word2index``) and index → word
# (``index2word``) dictionaries, as well as a count of each word
# ``word2count`` to use to later replace rare words.
#

SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


######################################################################
# The files are all in Unicode, to simplify we will turn Unicode
# characters to ASCII, make everything lowercase, and trim most
# punctuation.
#

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


######################################################################
# To read the data file we will split the file into lines, and then split
# lines into pairs. The files are all English → Other Language, so if we
# want to translate from Other Language → English I added the ``reverse``
# flag to reverse the pairs.
#

def readLangs(lang1, lang2, reverse=False):
    print("Reading langs...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs



In [56]:
######################################################################
# Since there are a *lot* of example sentences and we want to train
# something quickly, we'll trim the data set to only relatively short and
# simple sentences. Here the maximum length is 10 words (that includes
# ending punctuation) and we're filtering to sentences that translate to
# the form "I am" or "He is" etc. (accounting for apostrophes replaced
# earlier).
#

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p, max_length):
    return len(p[0].split(' ')) < max_length and \
        len(p[1].split(' ')) < max_length and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs, max_length, verbosity=0):
  if verbosity > 0:
    print("max_length={0}".format(max_length))
  return [pair for pair in pairs if filterPair(pair, max_length=max_length)]


######################################################################
# The full process for preparing the data is:
#
# -  Read text file and split into lines, split lines into pairs
# -  Normalize text, filter by length and content
# -  Make word lists from sentences in pairs
#
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

def prepareData_jhm(input_lang, output_lang, pairs,
                    max_length, verbosity=0):
    #input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    #print("Read %s sentence pairs" % len(pairs))
    if verbosity > 0:
      print("input pairs: {0}".format(len(pairs)))    
    #pairs = filterPairs(pairs, max_length=max_length, verbosity=verbosity)
    if verbosity > 0:
      print("Trimmed to {0} pairs".format(len(pairs)))
      print("Adding to lang")
    for pair in tqdm(pairs):
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
        #if i > 10000:
        #  break
    print("Counted words:")
    #print(input_lang.name, input_lang.n_words)
    #print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

#input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(len(pairs_part1))
if False:
  parts = prepareData_jhm(
        max_length =0,
        input_lang=input_lang_part1,
        output_lang=output_lang_part1,
        pairs=pairs_part1,
        verbosity=2)
  if len(parts) != 3:
    warnings.warn("Problem preparing the data")
  else:
    input_lang, output_lang, pairs = parts
  print(len(pairs))
  #random.choices(pairs, k=10)  

62357


In [59]:
verbosity = 2
max_length = 600
hidden_size = 300

encoder = EncoderRNN(input_lang_part1.n_words, hidden_size, verbosity=2).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang_part1.n_words, 
                         max_length=max_length, 
                         dropout_p=0.1, verbosity=2).to(device)
if verbosity > 0:
  print("encoder type= {0}".format(type(encoder)))
  print("decoder type= {0}".format(type(decoder)))
trainIters(encoder=encoder, decoder=decoder, n_iters=1000,
           input_lang=input_lang_part1,
           output_lang=output_lang_part1, 
           pairs=pairs_part1,
           max_length=max_length,
            print_every=1, plot_every=100,verbosity=2)

  0%|          | 0/350 [00:00<?, ?it/s]

encoder type= <class '__main__.EncoderRNN'>
decoder type= <class '__main__.AttnDecoderRNN'>
trainIters n_iters: 1000
Start iter dt:2020-12-14 05:43:11.618500
1 / 1000
train
  input_tensor.shape torch.Size([350, 1])
  target_tensor.shape: torch.Size([13, 1])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])



 34%|███▍      | 119/350 [00:00<00:08, 28.23it/s]

forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  h


  0%|          | 0/1000 [00:00<?, ?it/s]


forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  hidden.shape torch.Size([1, 1, 300])
forward input.shape torch.Size([1])  h

RuntimeError: ignored